## csv 데이터 기반 전처리

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

class AISPreprocessor:
    # data_dir: 경로 데이터 파일(csv)이 존재하는 폴더 이름 
    # input_seq_len: 참조할 이전 과거 정보(default: 10분)
    # output_seq_len: 예측할 미래 정보(default: 30초)
    def __init__(self, data_dir, input_seq_len=20, output_seq_len=1):
        self.data_dir = data_dir
        self.input_seq_len = input_seq_len
        self.output_seq_len = output_seq_len

        # 수동 설정된 범위로 MinMaxScaler 초기화
        # 정규화 범위 설정
        lat_range = (33.0, 38.0)
        lon_range = (124.0, 132.0)
        sog_range = (0.0, 100.0)
        cog_range = (0.0, 360.0)
        
        # MinMaxScaler 수동 설정
        self.scaler = MinMaxScaler()
        self.scaler.min_ = np.array([
            -lat_range[0] / (lat_range[1] - lat_range[0]),
            -lon_range[0] / (lon_range[1] - lon_range[0]),
            -sog_range[0] / (sog_range[1] - sog_range[0]),
            -cog_range[0] / (cog_range[1] - cog_range[0])
        ])
        self.scaler.scale_ = np.array([
            1 / (lat_range[1] - lat_range[0]),
            1 / (lon_range[1] - lon_range[0]),
            1 / (sog_range[1] - sog_range[0]),
            1 / (cog_range[1] - cog_range[0])
        ])
        self.scaler.feature_names_in_ = np.array(['위도', '경도', 'SOG', 'COG'])


    def load_and_preprocess(self):
        input_seqs = []
        output_seqs = []
        count = 1
        for file in os.listdir(self.data_dir):
            if file.endswith('.csv'):
                print(f"---------- {count}번째 파일 진행 중 ----------")
                count += 1
                df = pd.read_csv(os.path.join(self.data_dir, file), encoding='cp949')
                df = self._preprocess_single_file(df)
                in_seqs, out_seqs = self._extract_sequences(df)
                input_seqs.extend(in_seqs)
                output_seqs.extend(out_seqs)

        return np.array(input_seqs), np.array(output_seqs)

    def _preprocess_single_file(self, df):
        df = df[['일시', '위도', '경도', 'SOG', 'COG']].copy()
        df['일시'] = pd.to_datetime(df['일시'])
        # 일시 기준 데이터 sorting
        df = df.sort_values('일시')
        # NA 데이터 drop
        df = df.dropna()
        
        # 각 데이터를, 일시 기준 30초 별 보간법을 적용해 transform 적용 ------------------------------------
        # 예: 01:00:04, 01:00:06, 01:00:16 데이터 -> 평균 -> 01:00:30(1시 0분 30초)으로 한 샘플 생성
        # 에: 01:01:05, 01:01:12, 01:01:15, 01:01:24 데이터 -> 평균 -> 01:01:30(1시 1분 30초)으로 한 샘플 생성
        # 평균을 적용하는 데이터는 위도, 경도, sog, cog 데이터 
        df = df.set_index('일시').resample('30s').mean().interpolate()
        df = df.reset_index()
        # -----------------------------------------------------------------------------------------------
        
        # 목적지 좌표: 마지막 위치
        dest_lat = df['위도'].iloc[-1]
        dest_lon = df['경도'].iloc[-1]
        
        # feature engineering을 위한 feature 저장 
        df['dest_lat'] = dest_lat
        df['dest_lon'] = dest_lon
        return df

    def _extract_sequences(self, df):
        input_seqs = []
        output_seqs = []
    
        total_len = self.input_seq_len + self.output_seq_len
        for i in range(len(df) - total_len):
            input_window = df.iloc[i:i+self.input_seq_len]
            output_window = df.iloc[i+self.input_seq_len:i+total_len]
    
            # 입력: 위도, 경도, SOG, COG (정규화)
            input_scaled = self.scaler.transform(input_window[['위도', '경도', 'SOG', 'COG']])
            
             # 목적지 좌표 가져오기 (하나만, 파일 마지막 지점 기준)
            dest_lat = input_window['dest_lat'].iloc[0]
            dest_lon = input_window['dest_lon'].iloc[0]
    
            #  Δlat, Δlon 계산 (목적지 - 현재 위치) 계산 - 이 값들은 정규화 진행 x
            delta_lat = dest_lat - input_window['위도']
            delta_lon = dest_lon - input_window['경도']
            
            # 목적지까지 거리 계산 
            distance = np.sqrt(delta_lat ** 2 + delta_lon ** 2)

            # 입력 최종: [정규화된 위도, 경도, SOG, COG, distance]
            delta_coords = np.stack([distance], axis=1)
            input_seq = np.hstack([input_scaled, delta_coords])
            
            # 출력: 위도, 경도, SOG, COG (정규화)
            output_seq = self.scaler.transform(output_window[['위도', '경도', 'SOG', 'COG']])
    
            input_seqs.append(input_seq)
            output_seqs.append(output_seq)
    
        return input_seqs, output_seqs

## 경로 예측모델 

- 입력 시퀀스 (위도, 경도, SOG, COG, (목적지-현재위치 위도), (목적지-현재위치 경도))
  
      ↓
  
- Linear 프로젝션 (input_size → d_model)

  
      ↓

  
- Positional Encoding 추가

  
      ↓

  
- Transformer Encoder (Multi-head Self Attention, FeedForward)

  
      ↓

  
- Decoder (MLP): MLP 기반의 단순한 구조를 가짐.

  
      ↓

  
- 출력 (예: 다음 시점의 위도, 경도, SOG, COG)

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
input_csv_files = [
    'bm_input_seqs',
    'dp_input_seqs',
    'ij_input_seqs',
    'ud_input_seqs',
    'yu_input_seqs'
]
output_csv_files=[
    'bm_label_seqs',
    'dp_label_seqs',
    'ij_label_seqs',
    'ud_label_seqs',
    'yu_label_seqs'
]
input_seqs = []
label_seqs = []

for file in input_csv_files:
    file_path = "routes/" + file + ".csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, header=None)
        data = df.values
        data = np.array(data).reshape(-1, 40, 5)
        input_seqs.append([data])
        
for file in output_csv_files:
    file_path = "routes/" + file+ ".csv"
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, header=None)
        data = df.values
        data = np.array(data).reshape(-1, 1, 4)
        label_seqs.append([data])
input_seqs = np.hstack(input_seqs)
label_seqs = np.hstack(label_seqs)
input_seqs = np.array(input_seqs)
label_seqs = np.array(label_seqs)
input_seqs = input_seqs.reshape(-1, 40, 5)
label_seqs = label_seqs.reshape(-1, 1, 4)
 
print(input_seqs.shape)
print(label_seqs.shape)

(414877, 40, 5)
(414877, 1, 4)


In [3]:
print(input_seqs[1250:1251])
print(label_seqs[1250:1251])

[[[0.25588175 0.36696755 0.127      0.71375    0.77598338]
  [0.255807   0.36671021 0.1265     0.71763889 0.77462702]
  [0.2557355  0.36644599 0.12725    0.71638889 0.77322116]
  [0.25566583 0.36618552 0.1285     0.71930556 0.77183597]
  [0.25558975 0.36591823 0.129      0.71840278 0.7704326 ]
  [0.25552033 0.36565417 0.1305     0.71861111 0.76903127]
  [0.2554505  0.36538828 0.13075    0.72388889 0.76762353]
  [0.25537817 0.3651224  0.129      0.71875    0.7662273 ]
  [0.25530317 0.36485641 0.129      0.71444444 0.76484259]
  [0.2552275  0.36459417 0.1295     0.71347222 0.76348651]
  [0.255145   0.36433109 0.12875    0.71291667 0.76215107]
  [0.25506683 0.36407698 0.129      0.71291667 0.76085967]
  [0.25497975 0.3638075  0.12825    0.71118056 0.75950762]
  [0.2548975  0.36355562 0.1285     0.71097222 0.75825027]
  [0.25480467 0.36328818 0.129      0.70909722 0.75693704]
  [0.25471867 0.36302969 0.128      0.71375    0.75565942]
  [0.254633   0.3627675  0.12725    0.71104167 0.7543622

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class DestinationLoss(nn.Module):
    def __init__(self, weight_main=0.8):
        super().__init__()
        self.weight_main = weight_main
        
    def forward(self, y_pred, y_true):
        # [[]] 꼴로 변환
        y_pred = y_pred.squeeze(1)
        y_true = y_true.squeeze(1)
        # 예측 결과 분해
        pred_lat, pred_lon = y_pred[:, 0], y_pred[:, 1]
        true_lat, true_lon = y_true[:, 0], y_true[:, 1]
        pred_sog, pred_cog = y_pred[:, 2], y_pred[:, 3]
        true_sog, true_cog = y_true[:, 2], y_true[:, 3]

        # 위치 오차 (위경도 기준)
        loc_loss = F.mse_loss(pred_lat, true_lat) + F.mse_loss(pred_lon, true_lon)

        # SOG, COG 오차
        motion_loss = 0.1* F.mse_loss(pred_sog, true_sog) + 0.9*F.mse_loss(pred_cog, true_cog)

        # 최종 Loss: 위치 오차 + SOG, COG 오차
        total_loss = (
            self.weight_main * loc_loss +
            (1 - self.weight_main) * motion_loss
        )
        
        return total_loss

In [140]:
## 위치 정보 전달을 위한 정적 포지셔널 인코딩
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # even index
        pe[:, 1::2] = torch.cos(position * div_term)  # odd index
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x + self.pe[:, :x.size(1)]
        return x


### 시계열 학습을 위한 트랜스포머 회귀 모델
## d_model: 주목할 input의 특징들
## nhead: 멀티 헤드 어텐션 헤드 수
## dim_feedforward: FFN 차원 수
# dim_feedforward = d_model * 4
# d_model % n_head = 0
class TransformerPredictor(nn.Module):
    def __init__(self, input_size=5, output_size=4, d_model=128, nhead=8, num_layers=4, dim_feedforward=512, dropout=0.1, use_attention_pool=True):
        super(TransformerPredictor, self).__init__()
        # Attention pooling ------------------------------------------------------------------
        self.use_attention_pool = use_attention_pool
        if self.use_attention_pool:
            self.attn_pool = nn.Sequential(
                nn.Linear(d_model, 128),
                nn.Tanh(),
                nn.Linear(128, 1)  # 각 time step에 대한 score 출력
            )
        # --------------------------------------------------------------------------------------
        self.input_proj = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, d_model)
        )
        self.pos_encoder = PositionalEncoding(d_model) # 포지셔널 인코딩을 통해 순서 정보를 추가
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, # input의 특징들
            nhead=nhead, # 멀티 헤드 어텐션 헤드 수
            dim_feedforward=dim_feedforward, # FFN 차원 수, 기본 2048
            dropout=dropout,
            batch_first=True,
            activation="gelu", # default="relu"
        )
        # 인코더
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # MLP 기반 디코더
        self.decoder = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.GELU(),
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_size)
        )

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        x = self.input_proj(x)  # (batch_size, seq_len, d_model)
        x = self.pos_encoder(x)  # (batch_size, seq_len, d_model)
        x = self.transformer_encoder(x)  # (batch_size, seq_len, d_model)
        # Attention pooling 사용 ---------------------------------------------
        if self.use_attention_pool:
            # Attention score 계산
            attn_weights = self.attn_pool(x)  # (batch_size, seq_len, 1)
            attn_weights = torch.softmax(attn_weights, dim=1)  # normalize
            x_last = (attn_weights * x).sum(dim=1)  # 가중 평균
        else:
            # 기본 평균 풀링 사용
            x_last = x[:, -40:, :].mean(dim=1)
        out = self.decoder(x_last)  # (batch_size, output_size)
        # ---------------------------------------------------------------------
        # 차원을 맞추기 위해 seq_len=1 축을 다시 추가
        out = out.unsqueeze(1)  # (batch_size, 1, output_size)
        return out

In [141]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def train_transformer_model(model, train_data, val_data=None, num_epochs=50, batch_size=128, learning_rate=1e-3, device='cpu'):
    """
    model: TransformerPredictor 모델
    train_data: (x_train_tensor, y_train_tensor)
    val_data: (x_val_tensor, y_val_tensor)
    """
    model.to(device)
    
    ## train_data를 train / validation data로 분할 --------------------------------
    x_train, y_train = train_data
    x_train_f, x_val, y_train_f, y_val = train_test_split(
        x_train, y_train, test_size=0.1, random_state=42
    )

    
    train_dataset = TensorDataset(x_train_f, y_train_f)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    val_dataset = TensorDataset(x_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    # ----------------------------------------------------------------------------
    
    # Loss & Optimizer
    #criterion = nn.MSELoss()
    criterion = DestinationLoss(weight_main=0.99)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-2)
    # Learning-rate annealing
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

    for epoch in range(num_epochs):
        torch.cuda.empty_cache()
        model.train()
        total_loss = 0

        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    
            optimizer.zero_grad()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1) # gradient exploding 방지 
            outputs = model(batch_x)  # (batch, seq_len, output_size)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"[Epoch {epoch+1}/{num_epochs}] Train Loss: {avg_loss:.6f}")

        # Validation 진행
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for val_x, val_y in val_loader:
                val_x, val_y = val_x.to(device), val_y.to(device)
                val_outputs = model(val_x)
                val_loss = criterion(val_outputs, val_y)
                total_val_loss += val_loss.item()
        
            avg_val_loss = total_val_loss / len(val_loader)
            print(f"           ↳ Val Loss: {avg_val_loss:.6f}")

        scheduler.step()

In [ ]:
# 모델 생성
model = TransformerPredictor(input_size=5, output_size=4)
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# numpy → torch tensor로 변환
input_tensor = torch.tensor(input_seqs, dtype=torch.float32)
output_tensor = torch.tensor(label_seqs, dtype=torch.float32)
# 학습
train_transformer_model(model, (input_tensor, output_tensor), num_epochs=20, device="cuda" if torch.cuda.is_available() else "cpu")

## 자가회귀 예측 코드

In [ ]:
# 모델 저장
torch.save(model, "route_predictor15.pth")

In [ ]:
#model = torch.load("route_predictor11.pth", weights_only=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
import numpy as np

## 자가회귀를 위한 함수
## 선박의 예상 경로가 목적지 인근 부근이 될 때까지 모델의 예측 반복 / 임계값: max_steps
def predict_autoregressive(model, initial_seq, dest_lat, dest_lon, scaler, max_steps, distance_threshold=0.1):
    """
    model: 학습된 Transformer 모델
    initial_seq: 초기 입력 시퀀스 (torch.Tensor), shape: (1, 10, 4), 정규화된 값이 들어와야 됨.
    dest_lat, dest_lon: 목적지 좌표
    scaler: 학습에 사용한 MinMaxScaler
    device: 'cpu' or 'cuda'
    max_steps: 최대 예측 스텝 수
    distance_threshold: 도착지와 거리 차가 이 값 이하이면 도착으로 간주
    """
    model.eval()
    input_seq = initial_seq.clone()  # input_seq의 복사 생성
    all_preds = []

    for step in range(max_steps):
        input_tensor = input_seq
        input_tensor = input_tensor.to(device)
        # 예측

        with torch.no_grad():
            pred = model(input_tensor).squeeze(0).cpu().numpy()  # (seq_len, output_size)
    
        all_preds.append(pred)
        ## 예상 경로 도착지 도달 여부 확인 매커니즘 ----------------------------------------------------------
        # 예측한 경로값 역정규화
        pred_denorm = scaler.inverse_transform(pred.reshape(1, -1))[0][:4]
        pred_lat, pred_lon = pred_denorm[:2]
        # 디버깅 코드
        if(step % 50 == 0):
            print(f"[Step {step+1}] Predicted: ({pred_lat:.5f}, {pred_lon:.5f}) | "
                  f"Target: ({dest_lat:.5f}, {dest_lon:.5f}) | "
                  f"ΔLat: {abs(pred_lat - dest_lat):.5f}, ΔLon: {abs(pred_lon - dest_lon):.5f}")
    
    
        if abs(pred_lat - dest_lat) < distance_threshold and abs(pred_lon - dest_lon) < distance_threshold:
            print(f"🚢 목적지 도달 - Step: {step + 1}, {int(step/120)} 시간 {step%120} 분 소요")
            break

        ## ------------------------------------------------------------------------------------------------
            
        ## 입력 시퀀스 업데이트: 다음 입력을 만듦 -----------------------------------------------
        # 도착지 위도/경도 - 예측된 다음(1분 후) 위도/경도
        delta_lat = dest_lat - pred_lat
        delta_lon = dest_lon - pred_lon


        # 목적지까지 거리 계산 
        distance = np.sqrt(delta_lat ** 2 + delta_lon ** 2)
        
        # 정규화된 lat/lon/sog/cog (4개만 사용)
        pred_norm = pred.reshape(1, -1)[0]  # (4,)
        pred_norm = np.concatenate([pred_norm, [distance]])  # (5,)
        next_input = pred_norm
        # ---------------------------------------
        # 기존 시퀀스에서 가장 앞 데이터 제거, 새 데이터 추가
        input_seq_np = input_seq.squeeze(0).numpy()
        input_seq_np = np.vstack([input_seq_np[1:], next_input])
        input_seq = torch.tensor([input_seq_np], dtype=torch.float32)  # (1, 10, 5)
        # ----------------------------------------------------------------------
    return np.array(all_preds)

In [ ]:
from math import atan2, sqrt, degrees

## 초기 20분 시퀀스 (테스트 Input) ---------------------------------------------------------
pre = AISPreprocessor('rou/', 40, 1)
# 테스트 데이터 후보군 
#df = pd.read_csv('rou/dp_route_1_202002.csv', encoding='cp949', parse_dates=['일시'])
df = pd.read_csv('rou/ud_route_1_202002.csv', encoding='cp949', parse_dates=['일시'])
#df = pd.read_csv('rou/yu_route_1_202003.csv', encoding='cp949', parse_dates=['일시'])
#df = pd.read_csv('rou/bm_route_1_202009.csv', encoding='cp949', parse_dates=['일시'])
#df = pd.read_csv('rou/ij_route_1_202002.csv', encoding='cp949', parse_dates=['일시'])

# interpolation 수행
df = pre._preprocess_single_file(df)
initial_seq = df.iloc[:40]
# ----------------------------------------------------------------------------------------

# MinMaxScaler 수동 설정 ----------------------------------------------------------
# 정규화 범위 설정
lat_range = (33.0, 38.0)
lon_range = (124.0, 132.0)
sog_range = (0.0, 100.0)
cog_range = (0.0, 360.0)

input_scaler = MinMaxScaler()
input_scaler.min_ = np.array([
    -lat_range[0] / (lat_range[1] - lat_range[0]),
    -lon_range[0] / (lon_range[1] - lon_range[0]),
    -sog_range[0] / (sog_range[1] - sog_range[0]),
    -cog_range[0] / (cog_range[1] - cog_range[0])
])
input_scaler.scale_ = np.array([
    1 / (lat_range[1] - lat_range[0]),
    1 / (lon_range[1] - lon_range[0]),
    1 / (sog_range[1] - sog_range[0]),
    1 / (cog_range[1] - cog_range[0])
])
input_scaler.feature_names_in_ = np.array(['위도', '경도', 'SOG', 'COG'])
# --------------------------------------------------------------------------------

# 목적지 좌표 설정 --------------------
pohang = (36.0320, 129.3884)
donghae = (37.5340, 129.1161)
ulsan = (35.4985, 129.3850)
mokpo  = (34.7925, 126.3814)
jeju = (33.5136, 126.5230)

dest_lat = donghae[0]
dest_lon = donghae[1]
# ------------------------------------

# 입력 시퀀스 생성 
test_input_seq = []
for _, row in initial_seq.iterrows():
    # 1. 정규화된 위도, 경도, SOG, COG
    scaled = input_scaler.transform([[row['위도'], row['경도'], row['SOG'], row['COG']]])[0]
    # 2. Δlat, Δlon 계산
    delta_lat = dest_lat - row['위도']
    delta_lon = dest_lon - row['경도']
    
    # 3. distance 및 heading 계산
    distance = sqrt(delta_lat ** 2 + delta_lon ** 2)
    # 4. 최종 입력 벡터 구성 (5차원)
    input_row = list(scaled) + [distance]
    test_input_seq.append(input_row)

test_input_seq = torch.tensor([test_input_seq], dtype=torch.float32)  # (1, 40, 5)

# 목적지 좌표 설정
destination_lat = dest_lat
destination_lon = dest_lon
scaler = input_scaler

# 예측 실행
preds =  predict_autoregressive(
    model=model,                      # 학습된 Transformer 모델
    initial_seq=test_input_seq,       # 초기 입력 시퀀스 
    dest_lat=destination_lat,         # 목적지 위도
    dest_lon=destination_lon,         # 목적지 경도
    scaler=scaler,                    # 학습에 사용된 MinMaxScaler                 
    max_steps=2400                    # 예측할 시간 길이 (12시간)#
)

In [ ]:
def inverse_transform_preds(preds, scaler):
    """
    역정규화를 수행하여 원래의 값으로 변환
    preds: 예측된 값들 (numpy 배열), shape: (steps, output_size)
    scaler: 학습에 사용된 MinMaxScaler
    """
    # 위도, 경도, SOG, COG 값만 역정규화
    preds_unscaled = preds.copy()  # 예측된 값을 복사

    # 위도, 경도, SOG, COG를 역정규화
    preds_unscaled[:, :4] = scaler.inverse_transform(preds_unscaled[:, :4])  # 역정규화
    return preds_unscaled

In [ ]:
import folium
from folium.plugins import AntPath

def visualize_route(initial_seq, preds_inverse, dest_lat, dest_lon):
    """
    예측된 경로를 시각화하는 함수
    initial_seq: 초기 입력 시퀀스 (numpy 배열), shape: (10, 6)
    preds_inverse: 역정규화된 예측 결과, shape: (steps, 4)
    dest_lat, dest_lon: 목적지 좌표
    """
    # 초기 위치
    start = initial_seq[0, 0][:4]
    start = scaler.inverse_transform([start])
    start_lat = start[0][0] 
    start_lon = start[0][1]
    # 지도 생성 (출발지와 목적지가 모두 보이도록 설정)
    route_map = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # 시작점, 목적지 마커 추가
    folium.Marker([start_lat, start_lon], tooltip='Start', icon=folium.Icon(color='green')).add_to(route_map)
    folium.Marker([dest_lat, dest_lon], tooltip='Destination', icon=folium.Icon(color='red')).add_to(route_map)

    # 예측 경로
    route_coords = [[lat, lon] for lat, lon in preds_inverse[:, :2]]  # 위도, 경도만 사용
    # 예측 경로를 PolyLine으로 시각화
    folium.PolyLine(route_coords, color='blue', weight=3, tooltip="Predicted Route").add_to(route_map)

    # 예측 경로에 애니메이션 효과 추가
    AntPath(route_coords).add_to(route_map)

    return route_map

In [ ]:
# 예측 결과를 역정규화 후 시각화하는 전체 코드
def predict_and_visualize(model, initial_seq, dest_lat, dest_lon, scaler, max_steps=4800, distance_threshold=0.1):
    preds = predict_autoregressive(model, initial_seq, dest_lat, dest_lon, scaler, max_steps, distance_threshold)
    
    # 역정규화
    preds = preds.squeeze(1)
    preds_inverse = inverse_transform_preds(preds, scaler)
    # 예측 경로 시각화
    route_map = visualize_route(initial_seq.numpy(), preds_inverse, dest_lat, dest_lon)

    return route_map

In [ ]:
# 예시: 초기 시퀀스와 목적지 좌표로 예측 및 시각화
route_map = predict_and_visualize(model, test_input_seq, dest_lat=dest_lat, dest_lon=dest_lon, scaler=scaler)
route_map.save('predicted_route_map_bm.html')

### 학습 과정 시각화 코드

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# 에포크별 데이터
epochs = list(range(1, 11))
train_loss = [0.004675, 0.001154, 0.001223, 0.000743, 0.000353, 0.000292, 0.000247, 0.000241, 0.000229, 0.000241]
val_loss = [0.001190, 0.002310, 0.001023, 0.000420, 0.000270, 0.000283, 0.000213, 0.000214, 0.000234, 0.000254]

# DataFrame으로 변환
df = pd.DataFrame({
    'Epoch': epochs * 2,
    'Loss': train_loss + val_loss,
    'Type': ['Train'] * 10 + ['Validation'] * 10
})


In [ ]:
# 시각화
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, x='Epoch', y='Loss', hue='Type', marker='o')
plt.title('Training vs Validation Loss')
plt.grid(True)
plt.tight_layout()
plt.show()
